In [ ]:
!pip install fuzzywuzzy python-Levenshtein

In [ ]:
import pandas as pd
from collections import Counter
from fuzzywuzzy import fuzz

Read CIRCE query raw dataset

In [ ]:
df_circe = pd.read_csv("data/circe_queries_raw.csv", sep=";", dtype={"codice_C": "Int64", "coere5": "Int64"}
)

Select & rename columns

In [ ]:
columns_to_keep = [
    'RICHIESTA_UTENTE_SUPER_CLEAN',
    'NWORD',
    'codice_C',
    'coere5']

columns_to_rename = {
	'RICHIESTA_UTENTE_SUPER_CLEAN': 'query', 
    'NWORD': 'nword',
	'codice_C': 'codice_circe', 
    'coere5': 'coerenza'}
    
df_circe = df_circe[columns_to_keep].rename(columns=columns_to_rename)

In [ ]:
# Normalizzazione del testo
df_circe["query_clean"] = (
    df_circe["query"]
    .str.lower()
    .str.strip()
)

Filter rows where 'coerenza' is 1

In [ ]:
df_circe_filtered = df_circe[df_circe['coerenza'] == 1]

In [ ]:
df_circe_filtered.head()

In [ ]:
df_circe_filtered.value_counts('nword').sort_index()

In [ ]:
df_circe_filtered.value_counts('nword').sort_index().plot()

Find the nword value corresponding to THRESHOLD of observations

In [ ]:
THRESHOLD = 85

counts = df_circe_filtered['nword'].value_counts().sort_index()  # ordina per valore crescente di nword

# Totale delle osservazioni
total = counts.sum()

# Somma cumulativa
cum_counts = counts.cumsum()

# Calcolo soglia THRESHOLD
threshold = total * THRESHOLD / 100

# Trova il primo valore di nword per cui la somma cumulativa supera l’85%
nword_threshold = cum_counts[cum_counts >= threshold].index[0]

print(f"Il valore di nword che copre l'{THRESHOLD}% delle osservazioni è: {nword_threshold}")


In [ ]:
df_circe_filtered_threshold = df_circe_filtered[df_circe_filtered['nword'] <= nword_threshold]

In [ ]:
df_circe_filtered_threshold.value_counts('nword').sort_index()

In [ ]:
df_circe_filtered_threshold.shape

In [ ]:
# === 2. Funzione per clusterizzare testi simili ===
def clusterizza_testi(testi, soglia=80):
    testi = list(testi)  # non tolgo duplicati (servono per frequenza)
    unici = list(set(testi))
    cluster = []
    while unici:
        base = unici.pop(0)
        gruppo = [base]
        simili = [t for t in unici if fuzz.ratio(base, t) >= soglia]
        for s in simili:
            unici.remove(s)
            gruppo.append(s)
        cluster.append(gruppo)
    return cluster

In [ ]:
# === 3. Funzione per scegliere label rappresentativa ===
def scegli_label(gruppo, tutti_testi):
    counts = Counter([t for t in tutti_testi if t in gruppo])
    max_freq = max(counts.values())
    candidati = [t for t, c in counts.items() if c == max_freq]
    
    # ----------------------------------------------------------
    # Se più candidati → prendo quello più lungo (più leggibile)
    # DA RIVEDERE: forse meglio media lunghezza?
    # ----------------------------------------------------------
    return max(candidati, key=len)

In [ ]:
# === 4. Applico il clustering per ogni codice_circe e nword ===
risultati = []
for (codice, nword), subset in df_circe_filtered_threshold.groupby(["codice_circe", "nword"]):
    tutti_testi = subset["query_clean"].tolist()
    clusters = clusterizza_testi(tutti_testi, soglia=80)
    for i, gruppo in enumerate(clusters, start=1):
        label = scegli_label(gruppo, tutti_testi)
        for testo in gruppo:
            risultati.append({
                "codice_circe": codice,
                "nword": nword,
                "cluster_id": f"{codice}_{nword}_{i}",
                "testo": testo,
                "label_finale": label
            })


In [ ]:
# === 5. Salvo in due CSV ===
out_df = pd.DataFrame(risultati)
out_df.to_csv(f"output/cluster_output_threshold_{THRESHOLD}.csv", sep=";", index=False)

# File riepilogativo (senza ripetizioni)
final_df = out_df[["codice_circe", "nword", "cluster_id", "label_finale"]].drop_duplicates()
final_df.to_csv(f"output/cluster_summary_threshold_{THRESHOLD}.csv", sep=";", index=False)

print("Clustering completato:")
print("- cluster_output.csv -> tutti i testi con cluster")
print("- cluster_summary.csv -> riepilogo unico cluster")
